In [1]:
from ultralytics import YOLO
import numpy as np
import supervision as sv
import cv2
from scipy.spatial import cKDTree



In [2]:
def coloe_check(masked_img):
    hsv_boundary = {
        "white": np.array([[0, 15, 200], [75, 70, 255]], dtype=np.uint8),
        "red": np.array([[122, 63, 56], [132, 255, 255]], dtype=np.uint8),
        "orange": np.array([[100, 40, 170], [120, 255, 255]], dtype=np.uint8),
        "yellow": np.array([[80, 90, 100], [100, 255, 255]], dtype=np.uint8),
        "green": np.array([[50, 150, 60], [80, 210, 200]], dtype=np.uint8),
        "blue": np.array([[0, 70, 25], [40, 255, 210]], dtype=np.uint8)
    }
    color_deter = {
        "white": (255, 255, 255),
        "red": (0, 0, 255),
        "orange": (0, 165, 255),
        "yellow": (0, 255, 255),
        "green": (0, 128, 0),
        "blue": (255, 0, 0)
    }
    hsv = cv2.cvtColor(masked_img, cv2.COLOR_RGB2HSV)
    
    col_deter = {}
    for col_name, hsv_boun in hsv_boundary.items():
        check = cv2.inRange(hsv, hsv_boun[0], hsv_boun[1])
        pixel = cv2.countNonZero(check)
        col_deter[col_name] = pixel

    check_col = max(col_deter, key=col_deter.get)
    
    return color_deter[check_col]

In [3]:
def revise_points(approx_points, epsilon):
    pile = np.vstack([point for points in approx_points 
                      for point in points])
    kdtree = cKDTree(pile)
    combine_points = [] 
    ignore_point = set()
    for i, correct in enumerate(pile):
        if i not in ignore_point:
            
            region_indices = kdtree.query_ball_point(correct, r=epsilon)

            ignore_point.update(region_indices)

            ave_x = sum(pile[idx][0] for idx in region_indices) / len(region_indices)
            ave_y = sum(pile[idx][1] for idx in region_indices) / len(region_indices)
            
            combine_points.append((ave_x, ave_y))
    
    merge_lines = []
    for point_set in approx_points:
        for i in range(len(point_set)):

            establish_points = np.array(point_set[i])

            distance = np.linalg.norm(combine_points - establish_points, axis=1)
            nearest_points = np.argmin(distance)
            
            point_set[i] = combine_points[nearest_points]
            merge_lines.append(point_set)
    
    return merge_lines, combine_points


In [4]:
#圖片測試
ori = cv2.imread("block_2.jpg")
scaled = 600 / ori.shape[0]

img = cv2.resize(ori, None, fx=scaled, fy=scaled)
model = YOLO("yolov8n-seg-custom.pt")

predict = model(img)[0]
detections = sv.Detections.from_yolov8(predict)

# mask_col = detections.mask[0].astype(np.int32)

for mask_col in detections.mask:
    mask = np.where(mask_col != 0, 255, 0).astype(np.uint8)
    masked_img = cv2.bitwise_and(img, img, mask=mask)#遮罩後顏色方塊

    rgb = coloe_check(masked_img)
    #抓左上、右下
    indices = np.where(mask != 0)

    y1, y2 = min(indices[0]), max(indices[0])
    x1, x2 = min(indices[1]), max(indices[1])
    masked_area = masked_img[y1-5:y2+5, x1-5:x2+5]
    # 放大部分區域
    mask_scaled = 300 / masked_area.shape[0]#遮罩圖縮放
    height, width = masked_area.shape[:2]
    new_height, new_width = int(height * mask_scaled), int(width * mask_scaled)
    masked_img_resize = cv2.resize(masked_area, (new_width, new_height))

    kernel_close = np.ones((10,10),np.uint8)
    closed = cv2.morphologyEx(masked_img_resize, cv2.MORPH_CLOSE, kernel_close, iterations=1)
    opened = cv2.morphologyEx(closed, cv2.MORPH_OPEN, kernel_close, iterations=1)
    shine = cv2.convertScaleAbs(opened, None, 1, 10)#增強亮度
    detail = cv2.detailEnhance(shine, None, 10, 0.2)
    gray = cv2.cvtColor(detail, cv2.COLOR_RGB2GRAY)#轉灰
    blur = cv2.GaussianBlur(gray, (9, 9), 13)#模糊
    edge = cv2.edgePreservingFilter(blur, cv2.RECURS_FILTER, 100, 0.4)#強化邊緣 

    sobel_x = cv2.Sobel(edge, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(edge, cv2.CV_64F, 0, 1, ksize=3)
    sobel_img = np.sqrt(sobel_x**2 + sobel_y**2)
    sobel_img = np.uint8(sobel_img)

    kernel_canny = np.ones((3,3),np.uint8)  
    canny_img = cv2.Canny(sobel_img, 50, 100)
    canny_img = cv2.dilate(canny_img, kernel_canny, iterations=2)
    canny_img = cv2.erode(canny_img, kernel_canny, iterations=1)

    contours, _ = cv2.findContours(canny_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contour_max = max(contours, key=cv2.contourArea)
    epsilon_out = 0.05 * cv2.arcLength(contour_max, True)

    approx_points = []#
    contour_area = cv2.contourArea(contour_max)
    for i, contour in enumerate(contours):
        if i == 0:
            contour = contour_max
        if cv2.contourArea(contour) < 0.1 * contour_area:
            continue
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx_point = cv2.approxPolyDP(contour, epsilon, True)
        approx_points.append(approx_point)
    # 繪製近似多邊形輪廓
    # contour_img = np.zeros((masked_img_resize.shape[0], masked_img_resize.shape[1], 3), dtype=np.uint8)
    # for approx_contours in approx_points:
    #     contour_img = cv2.drawContours(contour_img, [approx_contours], -1, (255, 255, 255), 1)

    # approx_lines_img = np.zeros((masked_img_resize.shape[0], masked_img_resize.shape[1], 3), dtype=np.uint8)
    # for approx_lines in approx_points:
    #     approx_lines_img = cv2.polylines(approx_lines_img, [approx_lines], isClosed=True, color=(255,255,255), thickness=2)
    
    # for points in approx_points:
    #     for point in points:
    #         x, y = point[0]
    #         notmerge_points = cv2.circle(approx_lines_img, (x, y), 5, color=(255,0,0), thickness=-1)
    #__________________________________________________________________
    merge_lines, merge_points = revise_points(approx_points, epsilon=epsilon_out)
    draw_revise_points = np.zeros((masked_img_resize.shape[0], masked_img_resize.shape[1], 3), dtype=np.uint8)

    for revise_line in merge_lines:
        draw_lin = cv2.polylines(masked_img_resize, [revise_line], isClosed=True, color=(0,255,0), thickness=2)

    for point in merge_points:
        x, y = point
        x = int(x/mask_scaled) + x1-5
        y = int(y/mask_scaled) + y1-5
        cv2.circle(img, (x, y), 5, rgb, -1)
cv2.imshow('ori', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'shape'

In [30]:
#鏡頭測試
lens = cv2.VideoCapture(0)
model = YOLO("yolov8n-seg-custom.pt")

In [29]:
lens.release()

In [31]:

while lens.isOpened():
    ret, frame = lens.read()
    # ori = cv2.imread("block_2.jpg")
    scaled = 600 / frame.shape[0]

    img = cv2.resize(frame, None, fx=scaled, fy=scaled)

    predict = model(img)[0]
    # box = sv.BoxAnnotator(thickness=1, text_thickness=None, text_scale=None)    #繪製外框
    detections = sv.Detections.from_yolov8(predict)

    # mask_col = detections.mask[0].astype(np.int32)

    if detections.mask is not None and ret:
        for mask_col in detections.mask:
            mask = np.where(mask_col != 0, 255, 0).astype(np.uint8)
            masked_img = cv2.bitwise_and(img, img, mask=mask)#遮罩後顏色方塊

            rgb = coloe_check(masked_img)
            #抓左上、右下
            indices = np.where(mask != 0)
            y1, y2 = min(indices[0])-5, max(indices[0])+5
            x1, x2 = min(indices[1])-5, max(indices[1])+5
            masked_area = masked_img[y1:y2, x1:x2]
            # 放大部分區域
            if masked_area.shape[0] > 0 and masked_area.shape[1] > 0 and masked_area is not None:
                mask_scaled = 300 / masked_area.shape[0]#遮罩圖縮放
                height, width = masked_area.shape[:2]
                new_height, new_width = int(height * mask_scaled), int(width * mask_scaled)
                masked_img_resize = cv2.resize(masked_area, (new_width, new_height))

                kernel_close = np.ones((10,10),np.uint8)
                closed = cv2.morphologyEx(masked_img_resize, cv2.MORPH_CLOSE, kernel_close, iterations=1)
                opened = cv2.morphologyEx(closed, cv2.MORPH_OPEN, kernel_close, iterations=1)
                shine = cv2.convertScaleAbs(opened, None, 1, 10)#增強亮度
                detail = cv2.detailEnhance(shine, None, 10, 0.02)
                gray = cv2.cvtColor(detail, cv2.COLOR_RGB2GRAY)#轉灰
                blur = cv2.GaussianBlur(gray, (9, 9), 13)#模糊
                edge = cv2.edgePreservingFilter(blur, cv2.RECURS_FILTER, 100, 0.4)#強化邊緣 

                sobel_x = cv2.Sobel(edge, cv2.CV_64F, 1, 0, ksize=3)
                sobel_y = cv2.Sobel(edge, cv2.CV_64F, 0, 1, ksize=3)
                sobel_img = np.sqrt(sobel_x**2 + sobel_y**2)
                sobel_img = np.uint8(sobel_img)

                kernel_canny = np.ones((3,3),np.uint8)  
                canny_img = cv2.Canny(sobel_img, 50, 100)
                canny_img = cv2.dilate(canny_img, kernel_canny, iterations=2)
                canny_img = cv2.erode(canny_img, kernel_canny, iterations=1)

                contours, _ = cv2.findContours(canny_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                contour_max = max(contours, key=cv2.contourArea)
                epsilon_out = 0.05 * cv2.arcLength(contour_max, True)

                approx_points = []#
                contour_area = cv2.contourArea(contour_max)
                for i, contour in enumerate(contours):
                    if i == 0:
                        contour = contour_max
                    if cv2.contourArea(contour) < 0.1 * contour_area:
                        continue
                    epsilon = 0.015 * cv2.arcLength(contour, True)
                    approx_point = cv2.approxPolyDP(contour, epsilon, True)
                    approx_points.append(approx_point)
                # # 繪製近似多邊形輪廓
                # contour_img = np.zeros((masked_img_resize.shape[0], masked_img_resize.shape[1], 3), dtype=np.uint8)
                # for approx_contours in approx_points:
                #     contour_img = cv2.drawContours(contour_img, [approx_contours], -1, (255, 255, 255), 1)

                # approx_lines_img = np.zeros((masked_img_resize.shape[0], masked_img_resize.shape[1], 3), dtype=np.uint8)
                # for approx_lines in approx_points:
                #     approx_lines_img = cv2.polylines(approx_lines_img, [approx_lines], isClosed=True, color=(255,255,255), thickness=2)

                # for points in approx_points:
                #     for point in points:
                #         x, y = point[0]
                #         nonmerge_points = cv2.circle(approx_lines_img, (x, y), 5, color=(255,0,0), thickness=-1)
                #__________________________________________________________________
                merge_lines, merge_points = revise_points(approx_points, epsilon=epsilon_out)
                draw_revise_points = np.zeros((masked_img_resize.shape[0], masked_img_resize.shape[1], 3), dtype=np.uint8)

                for revise_line in merge_lines:
                    draw_lin = cv2.polylines(masked_img_resize, [revise_line], isClosed=True, color=(0,255,0), thickness=2)

                for point in merge_points:
                    x, y = point
                    x = int(x/mask_scaled) + x1
                    y = int(y/mask_scaled) + y1
                    draw_poi = cv2.circle(img, (x, y), 5, rgb, -1) 
            else:
                masked_img_resize = None
                print("NO PREDICTION") 
    else:
        print("NO TARGET")
    cv2.imshow("test", img)
    # key = cv2.waitKey(1)
    # if key == 27:
    #     break

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
# lens.release()
# cv2.waitKey(0)
cv2.destroyAllWindows()



0: 480x640 (no detections), 10.0ms
Speed: 3.0ms preprocess, 10.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 cube, 11.0ms
Speed: 2.0ms preprocess, 11.0ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)



NO TARGET
NO PREDICTION


0: 480x640 (no detections), 8.0ms
Speed: 2.0ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 10.0ms
Speed: 2.0ms preprocess, 10.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 cube, 9.0ms
Speed: 2.0ms preprocess, 9.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 cube, 9.0ms
Speed: 3.0ms preprocess, 9.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 cube, 9.0ms
Speed: 2.0ms preprocess, 9.0ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)



NO TARGET
NO TARGET
NO PREDICTION
NO PREDICTION
NO PREDICTION


0: 480x640 1 cube, 9.0ms
Speed: 2.0ms preprocess, 9.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 cube, 9.0ms
Speed: 2.0ms preprocess, 9.0ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 cube, 8.0ms
Speed: 2.0ms preprocess, 8.0ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 9.0ms
Speed: 2.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 cube, 8.0ms
Speed: 2.0ms preprocess, 8.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)


...

NO PREDICTION
NO PREDICTION
NO PREDICTION


Speed: 2.0ms preprocess, 24.0ms inference, 6.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 cube, 23.0ms
Speed: 3.0ms preprocess, 23.0ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 25.5ms
Speed: 3.0ms preprocess, 25.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 23.0ms
Speed: 3.0ms preprocess, 23.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)



NO PREDICTION
NO PREDICTION
NO TARGET
NO TARGET


0: 480x640 (no detections), 26.0ms
Speed: 3.0ms preprocess, 26.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 cube, 27.0ms
Speed: 3.0ms preprocess, 27.0ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 cube, 23.0ms
Speed: 2.0ms preprocess, 23.0ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 20.0ms
Speed: 3.0ms preprocess, 20.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)


NO TARGET
NO PREDICTION
NO PREDICTION
NO TARGET
